In [15]:
import os
from zipfile import ZipFile
work_dir = os.getcwd()  

In [16]:
train_dir = os.path.join(work_dir,'train')       #Path of train directory. 
test_dir = os.path.join(work_dir,'test')         #Path of test directory.
os.makedirs(train_dir, exist_ok=True)  #Creating a directory for training examples. 
os.makedirs(test_dir, exist_ok=True)   #Creating a directory for testing examples. 

In [17]:
train_ratio=0.8
test_ratio=0.2
dir_list=os.listdir(os.path.join(work_dir ,'105_classes_pins_dataset'))
print('The Number of Classes in the Dataset is:{}'.format(len(dir_list)))

The Number of Classes in the Dataset is:105


In [18]:
from shutil import copyfile
import numpy as np

source_dir = os.path.join(work_dir ,'105_classes_pins_dataset')        #Unzipped dataset directory 

dir_list = os.listdir(source_dir)

for folder in dir_list:
    data_dir = os.listdir(os.path.join(source_dir,folder))
    np.random.shuffle(data_dir)                            #Shuffling the examples of the classes inside the dataset.
    os.makedirs(os.path.join(train_dir , folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir , folder), exist_ok=True)
    train_data = data_dir[:int(len(data_dir)*train_ratio+1)] #Splitting the training dataset with respect to the train_ratio.
    test_data = data_dir[-int(len(data_dir)*test_ratio):]
  
    for image in train_data:
        copyfile(os.path.join(source_dir,folder,image) , os.path.join(train_dir,folder,image)) #Copying the Training files from dataset to training directory.
    
    for image in test_data:
       copyfile(os.path.join(source_dir,folder,image) , os.path.join(test_dir,folder,image))  #Copying the Training file

In [19]:
print("The Number of Classes in the Training Set:{}".format(len(os.listdir(train_dir))))
print("The Number Of Classes in the Testing Set:{}".format(len(os.listdir(test_dir))))
print('For Verification of the Split.....')
print("The Number of Examples in the Class='pins_Katherine Langford' in the Dataset Directory:{}".format(len(os.listdir(os.path.join(source_dir,'pins_Katherine Langford')))))
print("The Number of Examples in the Class='pins_Katherine Langford' in the Train Directory:{}".format(len(os.listdir(os.path.join(train_dir,'pins_Katherine Langford')))))
print("The Number of Examples in the Class='pins_Katherine Langford' in the Test Directory:{}".format(len(os.listdir(os.path.join(test_dir,'pins_Katherine Langford')))))


The Number of Classes in the Training Set:105
The Number Of Classes in the Testing Set:105
For Verification of the Split.....
The Number of Examples in the Class='pins_Katherine Langford' in the Dataset Directory:226
The Number of Examples in the Class='pins_Katherine Langford' in the Train Directory:216
The Number of Examples in the Class='pins_Katherine Langford' in the Test Directory:80


In [20]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
rows=224  #Number of Pixels in the Rows for Input. 
cols=224 #Number of Pixels in Columns for Output

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# import metric
from keras.metrics import categorical_crossentropy
# optimization method
from keras.optimizers import SGD

def AlexNet(input_shape=(rows,cols,3), output_shape=105):
    model = Sequential()

    # Layer 1
    model.add(Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 2
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layers 3-5
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layers 6-8
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_shape, activation='softmax'))

    # Compile the model
    opt = SGD(lr=0.01)  # Changed learning rate for better convergence
    model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['accuracy'])

    return model

In [ ]:
model = AlexNet()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       1

c:\Users\imoua\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
%load_ext tensorboard

In [21]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 rotation_range=40,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1)
                                 
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(rows,cols),
                                                  class_mode='categorical')

test_generator=test_datagen.flow_from_directory(test_dir,
                                                target_size=(rows,cols),
                                                class_mode='categorical')

Found 16930 images belonging to 105 classes.
Found 6638 images belonging to 105 classes.


In [ ]:
import datetime
import os

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("./logs",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
# Create early stopping (once our model stops improving, stop training)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3) # stops after 3 rounds of no improvements

In [ ]:
NUM_EPOCHS = 100 #@param {type:"slider", min:10, max:100, step:10}

In [ ]:
# Build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version.
  """
  # Create a model
  model = AlexNet()

  # Create new TensorBoard session everytime we train a model
  tensorboard = create_tensorboard_callback()

  # Fit the model to the data passing it the callbacks we created
  model.fit(x=train_generator,
            epochs=NUM_EPOCHS,
            validation_data=test_generator,
            validation_freq=1, # check validation metrics every epoch
            callbacks=[tensorboard, early_stopping])

  return model

In [ ]:
model = train_model()

Epoch 1/100
441/441 [==============================] - 163s 367ms/step - loss: 4.6501 - accuracy: 0.0110 - val_loss: 4.6373 - val_accuracy: 0.0127
Epoch 2/100
441/441 [==============================] - 118s 266ms/step - loss: 4.6410 - accuracy: 0.0120 - val_loss: 4.6321 - val_accuracy: 0.0136
Epoch 3/100
441/441 [==============================] - 116s 262ms/step - loss: 4.6356 - accuracy: 0.0136 - val_loss: 4.6246 - val_accuracy: 0.0179
Epoch 4/100
441/441 [==============================] - 117s 265ms/step - loss: 4.6323 - accuracy: 0.0122 - val_loss: 4.6223 - val_accuracy: 0.0127
Epoch 5/100
441/441 [==============================] - 117s 266ms/step - loss: 4.6281 - accuracy: 0.0147 - val_loss: 4.6184 - val_accuracy: 0.0188
Epoch 6/100
441/441 [==============================] - 119s 270ms/step - loss: 4.6221 - accuracy: 0.0121 - val_loss: 4.6110 - val_accuracy: 0.0199
Epoch 7/100
441/441 [==============================] - 116s 262ms/step - loss: 4.6152 - accuracy: 0.0141 - val_loss: 4

In [ ]:
model.save("./models/faces_classication.h5")

In [22]:
model = tf.keras.models.load_model("./models/faces_classication.h5")

In [24]:
path="./image/messi.jpg"  #Path to the target image to be predicted. 

classes=train_generator.class_indices
class_names=list(classes.keys())   #List of the class names
img = tf.keras.preprocessing.image.load_img(
    path, target_size=(rows, cols)
)

img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
img_array=img_array/255.

score = model.predict(img_array)

print(
    "The Person in the Image is {} ."
    .format(class_names[np.argmax(score)][5:].title())
)

1/1 [==============================] - 0s 47ms/step
The Person in the Image is Lionel Messi .
